In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import gc

os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/ID_Migration/By_Division'

In [3]:
ID_group_2019Q1=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/ID_Migration/BL_2019Q1_rewards_migration_rewards_data_JL_2019-05-17.csv",
                           dtype=str,usecols=['customer_id_hashed','Group_before_the_Quarter','Group_after_the_Quarter','Recent_Quarter_Shopping_Group','NewRewards','sales_recent_Quarter'])
ID_group_2019Q1['sales_recent_Quarter']=ID_group_2019Q1['sales_recent_Quarter'].astype(float)



In [5]:
product_taxonomy=pd.read_table("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20190701-134908-900.txt",dtype=str,sep="|")
product_taxonomy=product_taxonomy[['division_id','class_code_id','subclass_id']].drop_duplicates()
division_table=product_taxonomy.copy()

divison_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",dtype=str,sep="|")
product_taxonomy=pd.merge(product_taxonomy,divison_name,on="division_id",how="left")

In [4]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root, file)

In [26]:
file_list_daily_sales_2019Q1=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
file_list_daily_sales_2019Q1=[x for x in file_list_daily_sales_2019Q1 if "2019-02-09" not in x]
file_list_daily_sales_2019Q1=sorted([x for x in file_list_daily_sales_2019Q1 if "aily" in x])



df_files_2019Q1_daily=pd.DataFrame({"file_path":file_list_daily_sales_2019Q1})
df_files_2019Q1_daily['week_end_dt']=df_files_2019Q1_daily['file_path'].apply(lambda x: x.split("ks/MediaStorm_")[1][:10])

df_files_2019Q1_daily=df_files_2019Q1_daily[((df_files_2019Q1_daily['week_end_dt']>="2019-02-03") & (df_files_2019Q1_daily['week_end_dt']<="2019-05-04"))]
df_files_2019Q1_daily.shape


(12, 2)

In [27]:
df=pd.DataFrame({"file_path":"/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20190209.txt","week_end_dt":"2019-02-09"},index=[0])

df_files_2019Q1_daily=df_files_2019Q1_daily.append(df)
df_files_2019Q1_daily=df_files_2019Q1_daily.sort_values("week_end_dt")
df_files_2019Q1_daily=df_files_2019Q1_daily.reset_index()
del df_files_2019Q1_daily['index']
df_files_2019Q1_daily.shape

(13, 2)

In [30]:
samplerows=None
df_2019Q1_sales_all=pd.DataFrame()
df_2019Q1_sales_div=pd.DataFrame()
i_counter=0
for file in df_files_2019Q1_daily['file_path'].tolist():
    df=pd.read_table(file,sep="|",dtype=str,nrows=samplerows,usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','class_code_id','subclass_id','item_transaction_units','item_transaction_amt'])
    df=df[~pd.isnull(df['customer_id_hashed'])]
    
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df['item_transaction_units']=df['item_transaction_units'].astype(int)
    # Negative sales included in the transction count
    
    df_trans_all=df[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans_all=df_trans_all.groupby(['location_id','transaction_dt','customer_id_hashed'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})
    
    df_sales_all=df.groupby(['location_id','transaction_dt','customer_id_hashed'])['item_transaction_amt','item_transaction_units'].sum().reset_index().rename(columns={"item_transaction_amt":"sales","item_transaction_units":"units"})
    df_all=pd.merge(df_sales_all,df_trans_all,on=['location_id','transaction_dt','customer_id_hashed'])
    
    df_2019Q1_sales_all=df_2019Q1_sales_all.append(df)
  
    
    # division level
    df=pd.merge(df,division_table,on=['class_code_id','subclass_id'],how="left")
    df_trans_div=df[['location_id','transaction_dt','transaction_id','customer_id_hashed','division_id']].drop_duplicates()
    df_trans_div=df_trans_div.groupby(['location_id','transaction_dt','customer_id_hashed','division_id'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})
    
    df_sales_div=df.groupby(['location_id','transaction_dt','customer_id_hashed','division_id'])['item_transaction_amt','item_transaction_units'].sum().reset_index().rename(columns={"item_transaction_amt":"sales",'item_transaction_units':'units'})
    df_div=pd.merge(df_sales_div,df_trans_div,on=['location_id','transaction_dt','customer_id_hashed','division_id'])
    
    df_2019Q1_sales_div=df_2019Q1_sales_div.append(df_div)
    i_counter+=1
    print(datetime.datetime.now(),i_counter,file)
    
del df
del df_all
del df_div
gc.collect()

2019-07-29 12:04:01.674344 1 /home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20190209.txt
2019-07-29 12:06:40.557051 2 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-16/MediaStormDailySales20190219-113605-481.txt
2019-07-29 12:08:20.382560 3 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-23/MediaStormDailySales20190226-112611-940.txt
2019-07-29 12:10:37.514881 4 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-02/MediaStormDailySales20190305-112738-899.txt
2019-07-29 12:12:49.078038 5 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-09/MediaStormDailySales20190312-121205-996.txt
2019-07-29 12:15:17.367642 6 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-16/MediaStormDailySales20190319-112334-057.txt
2019-07-29 12:17:51.119534 7 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-23/MediaStormDailySales20190326-112740-903.txt
2019-07-29 12:20:26.833379 8 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-30/MediaStormDailySales2

189

In [42]:
print(len(df_2019Q1_sales_all['customer_id_hashed'].unique()))
print(len(df_2019Q1_sales_div['customer_id_hashed'].unique()))
print(df_2019Q1_sales_all.shape)
print(df_2019Q1_sales_div.shape)

df_2019Q1_sales_all.head(2)

8392050
8392050
(104376916, 8)
(36899319, 7)


,location_id,transaction_dt,transaction_id,customer_id_hashed,class_code_id,subclass_id,item_transaction_units,item_transaction_amt
2,1524,2019-02-04,6937,e61a8cb799cf05226370da0c38c421445de550174621e5...,11009,11,1,2.8
4,4687,2019-02-09,1447,e82a54b631ca4a5dc1f2d1f946fe216387e186f4518cde...,12004,17,1,1.0


In [43]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/ID_Migration/By_Division'

In [45]:

df_2019Q1_sales_all.to_csv("./Q4_rewards_sales_trans.csv",index=False)
del df_2019Q1_sales_all
gc.collect()

1085

In [49]:
ID_group_2019Q1_shopped=ID_group_2019Q1[ID_group_2019Q1['Recent_Quarter_Shopping_Group']=="Recent_Quarter_Shopped"]

In [65]:
ID_group_2019Q1_shopped.shape

(8392050, 6)

In [67]:
ID_group_2019Q1_shopped['customer_id_hashed'].nunique()

8392050

In [60]:
df_2019Q1_sales_div_agg=df_2019Q1_sales_div.groupby(['customer_id_hashed','location_id','division_id'])['sales','units','trans'].sum().reset_index()
df_2019Q1_sales_div_agg['store_type']=np.where(df_2019Q1_sales_div_agg['location_id']=="6990","Online","Instore")
df_2019Q1_sales_div_agg=df_2019Q1_sales_div_agg.groupby(['customer_id_hashed','store_type','division_id'])['sales','units','trans'].sum().reset_index()

In [63]:
df_2019Q1_sales_div_agg['store_type'].unique()

array(['Instore', 'Online'], dtype=object)

In [69]:
output_division_with_id['customer_id_hashed'].nunique()

8392050

In [90]:
# 1. by by division, by id group

output_division_with_id=df_2019Q1_sales_div_agg.groupby(['customer_id_hashed','division_id'])['sales','units','trans'].sum().reset_index()

output_division_with_id=pd.merge(output_division_with_id,ID_group_2019Q1_shopped,on="customer_id_hashed",how="outer")

print(output_division_with_id.shape)
output_division_with_id.head(4)


(23846961, 10)


,customer_id_hashed,division_id,sales,units,trans,Group_before_the_Quarter,Group_after_the_Quarter,sales_recent_Quarter,Recent_Quarter_Shopping_Group,NewRewards
0,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,1,8.81,11,3,Active,Active,31.59,Recent_Quarter_Shopped,Old_Rewards_Members
1,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,2,14.38,10,3,Active,Active,31.59,Recent_Quarter_Shopped,Old_Rewards_Members
2,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,6,8.40,3,1,Active,Active,31.59,Recent_Quarter_Shopped,Old_Rewards_Members
3,000007eccbfdc528a745aea90d34404b4fae9fbad2525f...,1,2.79,2,1,NotAvailable_Before_Quarter,Active,15.44,Recent_Quarter_Shopped,Recent_Quarter_New_Sign_Ups


In [91]:
output_division_with_id_nunique=output_division_with_id.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','Recent_Quarter_Shopping_Group','NewRewards','division_id'])['customer_id_hashed'].count().reset_index()

output_division_with_id_sum=output_division_with_id.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','Recent_Quarter_Shopping_Group','NewRewards','division_id'])['sales','units','trans'].sum().reset_index()

output_division_with_id=pd.merge(output_division_with_id_sum,output_division_with_id_nunique,on=['Group_before_the_Quarter','Group_after_the_Quarter','Recent_Quarter_Shopping_Group','NewRewards','division_id'],how="outer")
print(output_division_with_id.shape)
output_division_with_id.head(5)

(52, 9)


,Group_before_the_Quarter,Group_after_the_Quarter,Recent_Quarter_Shopping_Group,NewRewards,division_id,sales,units,trans,customer_id_hashed
0,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,1,9.873832e+07,50234361,9821063,4677595
1,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,10,1.714465e+05,587518,452241,284875
2,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,2,1.024063e+08,30493135,8192600,4270441
3,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,3,8.560242e+07,6128308,2578312,1833239
4,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,4,9.858662e+07,10937798,4348621,2829381


In [92]:
# 2. by by store types, by id group

output_storetype_with_id=df_2019Q1_sales_div_agg.groupby(['customer_id_hashed','store_type'])['sales','units','trans'].sum().reset_index()

output_storetype_with_id=pd.merge(output_storetype_with_id,ID_group_2019Q1_shopped,on="customer_id_hashed",how="outer")

print(output_storetype_with_id.shape)
output_storetype_with_id.head(4)


(8418891, 10)


,customer_id_hashed,store_type,sales,units,trans,Group_before_the_Quarter,Group_after_the_Quarter,sales_recent_Quarter,Recent_Quarter_Shopping_Group,NewRewards
0,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,Instore,31.59,24,7,Active,Active,31.59,Recent_Quarter_Shopped,Old_Rewards_Members
1,000007eccbfdc528a745aea90d34404b4fae9fbad2525f...,Instore,15.44,9,3,NotAvailable_Before_Quarter,Active,15.44,Recent_Quarter_Shopped,Recent_Quarter_New_Sign_Ups
2,000008374518306e87838f387eedc47d49a447707dc6da...,Instore,49.77,7,3,Active,Active,49.77,Recent_Quarter_Shopped,Old_Rewards_Members
3,0000095241ba6a94ec0d4b481d2fafd22598cf8cad3399...,Instore,875.50,6,2,NotAvailable_Before_Quarter,Active,875.50,Recent_Quarter_Shopped,Old_Rewards_Members


In [93]:
output_storetype_with_id_nunique=output_storetype_with_id.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','Recent_Quarter_Shopping_Group','NewRewards','store_type'])['customer_id_hashed'].count().reset_index()

output_storetype_with_id_sum=output_storetype_with_id.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','Recent_Quarter_Shopping_Group','NewRewards','store_type'])['sales','units','trans'].sum().reset_index()

output_storetype_with_id=pd.merge(output_storetype_with_id_sum,output_storetype_with_id_nunique,on=['Group_before_the_Quarter','Group_after_the_Quarter','Recent_Quarter_Shopping_Group','NewRewards','store_type'],how="outer")
print(output_storetype_with_id.shape)
output_storetype_with_id.head(5)

(12, 9)


,Group_before_the_Quarter,Group_after_the_Quarter,Recent_Quarter_Shopping_Group,NewRewards,store_type,sales,units,trans,customer_id_hashed
0,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,Instore,5.795814e+08,109951592,31793738,6319442
1,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,Online,5.099926e+06,132598,49604,29496
2,Active,Active,Recent_Quarter_Shopped,Recent_Quarter_New_Sign_Ups,Instore,3.211113e+04,4407,1266,173
3,Active,Active,Recent_Quarter_Shopped,Recent_Quarter_New_Sign_Ups,Online,1.499700e+02,4,1,1
4,Lapsed_13_48,Active,Recent_Quarter_Shopped,Old_Rewards_Members,Instore,5.044978e+07,5829584,1843886,595549


In [94]:
# 3. by by store types, by division, by id group

output_storetype_division_with_id=df_2019Q1_sales_div_agg.groupby(['customer_id_hashed','store_type','division_id'])['sales','units','trans'].sum().reset_index()

output_storetype_division_with_id=pd.merge(output_storetype_division_with_id,ID_group_2019Q1_shopped,on="customer_id_hashed",how="outer")

print(output_storetype_division_with_id.shape)
output_storetype_division_with_id.head(4)


(23869633, 11)


,customer_id_hashed,store_type,division_id,sales,units,trans,Group_before_the_Quarter,Group_after_the_Quarter,sales_recent_Quarter,Recent_Quarter_Shopping_Group,NewRewards
0,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,Instore,1,8.81,11,3,Active,Active,31.59,Recent_Quarter_Shopped,Old_Rewards_Members
1,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,Instore,2,14.38,10,3,Active,Active,31.59,Recent_Quarter_Shopped,Old_Rewards_Members
2,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,Instore,6,8.40,3,1,Active,Active,31.59,Recent_Quarter_Shopped,Old_Rewards_Members
3,000007eccbfdc528a745aea90d34404b4fae9fbad2525f...,Instore,1,2.79,2,1,NotAvailable_Before_Quarter,Active,15.44,Recent_Quarter_Shopped,Recent_Quarter_New_Sign_Ups


In [95]:
output_storetype_division_with_id_nunique=output_storetype_division_with_id.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','Recent_Quarter_Shopping_Group','NewRewards','store_type','division_id'])['customer_id_hashed'].count().reset_index()

output_storetype_division_with_id_sum=output_storetype_division_with_id.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','Recent_Quarter_Shopping_Group','NewRewards','store_type','division_id'])['sales','units','trans'].sum().reset_index()

output_storetype_division=pd.merge(output_storetype_division_with_id_sum,output_storetype_division_with_id_nunique,on=['Group_before_the_Quarter','Group_after_the_Quarter','Recent_Quarter_Shopping_Group','NewRewards','store_type','division_id'],how="outer")
print(output_storetype_division.shape)
output_storetype_division.head(5)

(83, 10)


,Group_before_the_Quarter,Group_after_the_Quarter,Recent_Quarter_Shopping_Group,NewRewards,store_type,division_id,sales,units,trans,customer_id_hashed
0,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,Instore,1,9.862407e+07,50230719,9819596,4676893
1,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,Instore,10,1.714465e+05,587518,452241,284875
2,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,Instore,2,1.023910e+08,30491813,8192075,4270206
3,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,Instore,3,8.302622e+07,6083013,2558605,1826092
4,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,Instore,4,9.775929e+07,10881490,4332178,2823183


In [96]:
output_storetype_division.shape

(83, 10)

In [97]:
#### 
writer=pd.ExcelWriter("./BL_2019Q1_rewards_summary_div_store_type_groups_data_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
output_division_with_id.to_excel(writer,"output_division",index=False)
output_storetype_with_id.to_excel(writer,"output_store",index=False)
output_storetype_division.to_excel(writer,"output_storetype_division",index=False)
writer.save()